# Common utitities for all production alphas

In [ ]:
function get_recommendee_username()
    dir = "../../data/recommendations"
    open("$dir/recommendee_username.txt") do f
        read(f, String)
    end    
end;

In [ ]:
name = "../../data/recommendations/$(get_recommendee_username())/alphas/$source"
residual_alphas = [];

In [ ]:
using DataFrames

In [ ]:
@nbinclude("../TrainingAlphas/Alpha.ipynb");

In [ ]:
function get_recommendee_list()
    username = get_recommendee_username()
    dir = "../../data/recommendations"
    file = "$dir/$username/splits.jld2"
    load(file)["user_anime_list"]
end

function get_implicit_list()
    username = get_recommendee_username()
    dir = "../../data/recommendations"
    file = "$dir/$username/splits.jld2"
    load(file)["implicit_anime_list"]    
end

function get_split(split)
    @assert split in
            ["training", "validation", "test", "recommendee", "recommendee_inference"]
    if split == "recommendee"
        return get_recommendee_list()
    elseif split == "recommendee_inference"
        df = get_recommendee_list()
        return RatingsDataset(
            fill(df.user[1], num_items()),
            collect(1:num_items()),
            zeros(num_items()),
        )
    end
    file = "../../data/splits/splits.jld2"
    load(file, split)
end;

In [ ]:
function write_recommendee_alpha(model; outdir = source)
    items = 1:num_items()
    training = get_recommendee_list()
    users = fill(training.user[1], length(items))
    alpha = RatingsDataset(users, items, model(items))

    dir = "../../data/recommendations"
    username = get_recommendee_username()
    outdir = mkpath("$dir/$username/alphas/$outdir")
    save("$outdir/alpha.jld2", Dict("alpha" => alpha))
end;

In [ ]:
function read_recommendee_alpha(alpha)
    dir = "../../data/recommendations"
    username = get_recommendee_username()
    load("$dir/$username/alphas/$alpha/alpha.jld2")["alpha"]
end;

function get_alpha(alpha, split)
    @assert split in
            ["training", "validation", "test", "recommendee", "recommendee_inference"]
    if split == "recommendee"
        mask = get_split("recommendee").item
        df = read_recommendee_alpha(alpha)
        return RatingsDataset(df.user[mask], df.item[mask], df.rating[mask])
    elseif split == "recommendee_inference"
        return read_recommendee_alpha(alpha)
    end
    file = "../../data/alphas/$(alpha)/predictions.jld2"
    load(file, split)
end;